In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

In [ ]:
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)    
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)    
    print()

In [ ]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
sentence = "potato chips, cola"

sentence_encoded = tokenizer(sentence, return_tensors='pt')
sentence_decoded = tokenizer.decode(
  sentence_encoded["input_ids"][0],
  skip_speical_tokens=True
)

print('ENCODED SENTENCE:')
# here its accessing an object and then a nested array.
# didn't make sense to me first time.
print(sentence_encoded['input_ids'][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

In [ ]:
# PLAIN ZERO SHOT INFERENCE
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)    
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)    
    print(f'MODEL GENERATION — WiTHOUT PROMPT ENGINEERING:\n{output}\n')

In [ ]:
# ZERO SHOT INFERENCE with INSTRUCTION
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """
    
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)    
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)    
    print(f'MODEL GENERATION — WiTHOUT PROMPT ENGINEERING:\n{output}\n')

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}

"""
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']
        
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

In [ ]:
## example number 40
example_indices_full = [40]
# example number 200, it could be 30, 10, anything in the limits of the set
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

In [ ]:
# ONE SHOT INFERENCE
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)    
print(f'MODEL GENERATION — WITH PROMPT ENGINEERING:\n{output}\n')

In [ ]:
# example 40, 80, and 120
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

In [ ]:
# FEW SHOT INFERENCE
generation_config = GenerationConfig(max_new_tokens=50)
# Adjust temperature to mess with how random the completion is.
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=2.0)

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)    
print(f'MODEL GENERATION — WITH PROMPT ENGINEERING:\n{output}\n')

In [ ]:
print("ITS FINE TUNING TIME!!!")